In [49]:
import numpy as np

N = 10
J = 8
alpha = np.random.rand(J//2) + 1.0j * np.random.rand(J//2)
alpha = np.append(alpha, np.conj(alpha))
beta = np.random.rand(J//2) + 1.0j * np.random.rand(J//2)
beta = np.append(beta, np.conj(beta))
t = np.sort(np.random.uniform(0, 100, N))
y0 = np.sin(t)
u = alpha*np.exp(-beta * t[:, None])
v = np.exp(beta * t[:, None])
diag = 0.01 + np.sum(alpha).real + np.zeros(N)

K = np.sum(alpha*np.exp(-beta*np.abs(t[:, None] - t[None, :])[:, :, None]), axis=-1)
K[np.diag_indices_from(K)] = diag

K0 = np.tril(np.dot(u, v.T), -1) + np.triu(np.dot(v, u.T), 1)
K0[np.diag_indices_from(K0)] = diag
print("Semiseparable error: {0}".format(np.max(np.abs(K - K0))))

# Cholesky method
dt = np.diff(t)
phi = np.exp(-beta * dt[:, None])
D = np.empty(N, dtype=alpha.dtype)
X = np.empty((N, J), dtype=alpha.dtype)

# Explicit first step
# D[0] = np.sqrt(diag[0])
D[0] = diag[0]
X[0] = 1.0 / D[0]
S = X[0][:, None] * X[0][None, :] * D[0]

# Then the rest
for n in range(1, N):
    St = phi[n-1][:, None] * phi[n-1][None, :] * S
#     D[n] = np.sqrt(diag[n] - np.sum(alpha[None, :] * alpha[:, None] * St))
    D[n] = diag[n] - np.sum(alpha[None, :] * alpha[:, None] * St)
    X[n] = (1.0 - np.sum(alpha[None, :] * St, axis=1)) / D[n]
    S = St + X[n][:, None] * X[n][None, :] * D[n]

# Check factorization
L = np.tril(np.dot(u, (v*X).T), -1)
L[np.diag_indices_from(L)] = 1.0
# L[np.diag_indices_from(L)] = D

print("Cholesky error: {0}".format(np.max(np.abs(np.dot(L, np.dot(np.diag(D), L.T)) - K))))
print(np.sum(np.log(D)).real - np.linalg.slogdet(K)[1])

Semiseparable error: 2.5757220274214744e-14
Cholesky error: 4.352092058577034e-14
5.3290705182e-15


In [56]:
y = np.array(y0)
z = np.empty(N, dtype=alpha.dtype)
z[0] = y[0] / D[0]
f = 0.0
for n in range(1, N):
    f = phi[n-1] * (f + alpha * X[n-1] * z[n-1] * D[n-1]) 
    z[n] = (y[n] - np.sum(f)) / D[n]
print("Forward sub error: {0}".format(np.max(np.abs(z - np.linalg.solve(L, y) / D))))

y = np.array(z)  # / D
z = np.empty(N, dtype=alpha.dtype)
z[-1] = y[-1]  # / D[-1]
f = 0.0
for n in range(N-2, -1, -1):
    f = phi[n] * (f + alpha * z[n+1]) 
    z[n] = (y[n] - np.sum(f * X[n]))  # / D[n]
print("Backward sub error: {0}".format(np.max(np.abs(z - np.linalg.solve(L.T, y)))))

print("Full solve error: {0}".format(np.max(np.abs(np.linalg.solve(K, y0) - z))))

Forward sub error: 2.692304429591852e-15
Backward sub error: 7.771642058341631e-16
Full solve error: 4.51055282916621e-16


In [3]:
a = 2*alpha[:J//2].real
b = 2*alpha[:J//2].imag
c = beta[:J//2].real
d = beta[:J//2].imag
tt = t[:, None]
u2 = np.concatenate((
    a*np.exp(-c*tt)*np.cos(d*tt),
    a*np.exp(-c*tt)*np.sin(d*tt),
    b*np.exp(-c*tt)*np.sin(d*tt),
    b*np.exp(-c*tt)*np.cos(d*tt),
), axis=1)
v2 = np.concatenate((
    np.exp(c*tt)*np.cos(d*tt),
    np.exp(c*tt)*np.sin(d*tt),
    np.exp(c*tt)*np.cos(-d*tt),
    np.exp(c*tt)*np.sin(-d*tt),
), axis=1)
K2 = np.tril(np.dot(u2, v2.T), -1) + np.triu(np.dot(v2, u2.T), 1)
K2[np.diag_indices_from(K2)] = diag
print("Semiseparable error: {0}".format(np.max(np.abs(K.real - K2))))

Semiseparable error: 4.218847493575595e-15


In [37]:
dt = np.diff(t)
phi = np.exp(-c * dt[:, None])

D = np.empty(N)
X = np.empty((N, 2*J))

D[0] = np.sqrt(diag[0])
X[0] = 1.0 / D[0]
S = X[0][:, None] * X[0][None, :]

for n in range(1, N):
    u = a * phi[n-1] * np.array([np.cos(d * t[n]), np.sin(d * t[n]), np.sin(d * t[n]), np.cos(d * t[n])])
    v = a * phi[n-1] * np.array([np.cos(d * t[n]), np.sin(d * t[n]), np.cos(d * t[n]), -np.sin(d * t[n])])

    D[n] = np.sqrt(diag[n] - np.sum((u.flatten()[:, None] * S * u.flatten()[None, :])))
    X[n] = (v.flatten() - np.sum((u * phi[n-1]).flatten() * S, axis=0)) / D[n]
    S = phi[n-1][:, None] * X[n] * phi[n-1][None, :]
    break
    D[n] = np.sqrt()

ValueError: operands could not be broadcast together with shapes (4,16) (1,4) 